# **Motive**

*This notebook is made to evaluate inference time of TIM-Net.*

## **TPU Access (Colab)**

In [1]:
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

Tensorflow version 2.12.0
Running on TPU  ['10.11.151.122:8470']


## **Pre-requisites**

In [2]:
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## **Imports**

In [45]:
import os
import sys
import gdown
import shutil
import fileinput
import subprocess
import tensorflow as tf
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.layers import (Activation, AveragePooling1D, Conv1D, Dense, Dropout, Embedding, Flatten, Input, LSTM, MaxPooling1D)
from keras.models import Model, Sequential
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical

## **Google Drive**

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Prepare project content**

In [39]:
# Create directories
def create_directories(directories):
    for directory in directories:
        try:
            os.makedirs(directory)
            print(f"Directory '{directory}' created")
        except FileExistsError:
            print(f"Directory '{directory}' already exists")
create_directories(['libs',  'models', 'weights', 'data', 'data/SalsaSound', 'data/MFCC'])

# Copy data files
try:
  shutil.copy('drive/MyDrive/SA-Speech/data/SalsaSound/commentary_short.wav', 'data/SalsaSound/commentary_short.wav')
  print('Copied commentary_short.wav')
  shutil.copy('drive/MyDrive/SA-Speech/data/MFCC/RAVDE.npy', 'data/MFCC/RAVDE.npy')
  print('Copied RAVDE.npy')
except FileExistsError:
  pass
except Exception as e:
  print("An error occurred while copying the SalsaSound data files:", e)

# Copy weights files
try:
  shutil.copy('drive/MyDrive/SA-Speech/weights/10-fold_weights_best_1.hdf5', 'weights/10-fold_weights_best_1.hdf5')
  print('Copied 10-fold_weights_best_1.hdf5')
  shutil.copy('drive/MyDrive/SA-Speech/weights/10-fold_weights_best_2.hdf5', 'weights/10-fold_weights_best_2.hdf5')
  print('Copied 10-fold_weights_best_2.hdf5')
  shutil.copy('drive/MyDrive/SA-Speech/weights/10-fold_weights_best_3.hdf5', 'weights/10-fold_weights_best_3.hdf5')
  print('Copied 10-fold_weights_best_3.hdf5')
  shutil.copy('drive/MyDrive/SA-Speech/weights/10-fold_weights_best_4.hdf5', 'weights/10-fold_weights_best_4.hdf5')
  print('Copied 10-fold_weights_best_4.hdf5')
except FileExistsError:
  pass
except Exception as e:
  print("An error occurred while copying the TIM-Net saved weights files:", e)

# Gathering external libraries
!git clone https://github.com/Jiaxin-Ye/TIM-Net_SER.git libs/TIM-Net

Directory 'libs' already exists
Directory 'models' already exists
Directory 'weights' already exists
Directory 'data' already exists
Directory 'data/SalsaSound' already exists
Directory 'data/MFCC' already exists
Copied commentary_short.wav
Copied RAVDE.npy
Copied 10-fold_weights_best_1.hdf5
Copied 10-fold_weights_best_2.hdf5
Copied 10-fold_weights_best_3.hdf5
Copied 10-fold_weights_best_4.hdf5
fatal: destination path 'libs/TIM-Net' already exists and is not an empty directory.


## **Set up saved TIM-Net**

In [47]:
sys.path = [path for path in sys.path if path != "/content/libs/TIM-Net"] # Use this to clean path of any redundancies
sys.path.append('/content/libs/TIM-Net')
import Code

file_path = 'libs/TIM-Net/Code/main.py'
line_numbers = [61]
new_contents = ['data = np.load("data/MFCC/"+args.data+".npy",allow_pickle=True).item()']

def change_line_content(file_path, line_numbers, new_contents):
  for line in fileinput.input(file_path, inplace=True):
    line_number = fileinput.filelineno()
    if line_number in line_numbers:
      line_index = line_numbers.index(line_number)
      line = new_contents[line_index] + '\n'
    print(line, end='')

change_line_content(file_path, line_numbers, new_contents)

try:
  !python libs/TIM-Net/Code/main.py --mode test --data RAVDE --test_path weights --split_fold 10 --random_seed 46 --model_path models
except:
  pass

2023-05-23 12:34:27.139677: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
###gpus:[]
TIMNET MODEL SHAPE: (215, 39)
Input Shape= (None, 215, 39)
Temporal create succes!
5/5 [==============================] - 3s 36ms/step - loss: 0.4587 - accuracy: 0.9306
1_Model evaluation:  [0.45871469378471375, 0.9305555820465088]    Now ACC: 93.06
5/5 [==============================] - 2s 31ms/step
              precision    recall  f1-score   support

       angry       0.89      1.00      0.94        24
        calm       0.84      1.00      0.91        21
     disgust       1.00      1.00      1.00        12
        fear       0.94      0.85      0.89        20
       happy       1.00      0.94      0.97        18
     neutral       0.90      0.75      0.82        12
         sad       0.93      0.93      0.93        15
    surprise       1.00      0.91      0.95        22

    accuracy                           0.93       144
   macro avg       0.